In [1]:
import tensorflow as tf
from model_v2 import LanguageModel
import json
import numpy as np
import re
import pandas as pd
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [4]:
with open('baomoi_punc/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('baomoi_punc/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)
idx2word = {k: v for v, k in word2idx.items()}
idx2char = {k: v for v, k in char2idx.items()}

In [5]:
with open('14/checkpoints/model_configs.json', 'r') as inp:
    params = json.load(inp)

In [6]:
model = LanguageModel(**params, is_training=False)

In [7]:
model.build_model()
saver = tf.train.Saver([x for x in tf.global_variables() if x not in tf.get_collection('LSTM_SAVED_STATE')])
sess.run(tf.global_variables_initializer())

In [8]:
saver.restore(sess, 'drive/latest/model.cpkt-181370')

INFO:tensorflow:Restoring parameters from drive/latest/model.cpkt-181370


In [9]:
import numpy as np

In [86]:
all_data = np.load('technology.npy')
all_text = [x[:min(400, len(x))] for x,_ in all_data]

In [87]:
all_text = [x for x in all_text if len(x) > 0]

In [88]:
all_vector = []
for i, p in enumerate(all_text):
    vector = sess.run(model.rnn_outputs, feed_dict={model.inputs: np.expand_dims(np.array(p), -1), model.reset_state: True, model.seq_lens: [len(p)]})
    all_vector.append(vector)
    print(i, len(all_data) - i)


0 839
1 838
2 837
3 836
4 835
5 834
6 833
7 832
8 831
9 830
10 829
11 828
12 827
13 826
14 825
15 824
16 823
17 822
18 821
19 820
20 819
21 818
22 817
23 816
24 815
25 814
26 813
27 812
28 811
29 810
30 809
31 808
32 807
33 806
34 805
35 804
36 803
37 802
38 801
39 800
40 799
41 798
42 797
43 796
44 795
45 794
46 793
47 792
48 791
49 790
50 789
51 788
52 787
53 786
54 785
55 784
56 783
57 782
58 781
59 780
60 779
61 778
62 777
63 776
64 775
65 774
66 773
67 772
68 771
69 770
70 769
71 768
72 767
73 766
74 765
75 764
76 763
77 762
78 761
79 760
80 759
81 758
82 757
83 756
84 755
85 754
86 753
87 752
88 751
89 750
90 749
91 748
92 747
93 746
94 745
95 744
96 743
97 742
98 741
99 740
100 739
101 738
102 737
103 736
104 735
105 734
106 733
107 732
108 731
109 730
110 729
111 728
112 727
113 726
114 725
115 724
116 723
117 722
118 721
119 720
120 719
121 718
122 717
123 716
124 715
125 714
126 713
127 712
128 711
129 710
130 709
131 708
132 707
133 706
134 705
135 704
136 703
137 702
138 70

In [89]:
all_vector2 = []
all_data2 = []
for v, m in zip(all_vector,[x for x in all_data if len(x[0]) > 0]):
    v = np.squeeze(v)
    if len(v) == 0:
        continue
    if v.ndim < 2:
        continue
    me = np.mean(v, axis=0, keepdims=False)
    l = v[-1, :]
    ma = np.max(v, axis=0, keepdims=False)
    all_vector2.append(np.concatenate((l, me, ma), axis=-1))
    all_data2.append(m)

In [90]:
df = pd.DataFrame()
# df['class'] = [x[1] for x in all_data2]
df['feat'] = [x for x in all_vector2 if isinstance(x, np.ndarray) and (not np.any(np.isnan(x)))]

In [91]:
df['message'] = [x for _, x in all_data2]

In [63]:
df.to_pickle('vector_technology.pkl')

In [ ]:
df = pd.read_pickle('vector_df.pkl')

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.svm import OneClassSVM

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.stack(df['feat']), df['class'], test_size=0.33, random_state=42)

In [108]:
clf = OneClassSVM(kernel='linear')
clf.fit(np.stack(df['feat']))

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.5, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

In [109]:
def predict_class(text):
    import unicodedata
    p = re.sub('\W+', '', re.sub('\d+','N',re.sub('[ ]+','_',re.sub('[\n\r]+','_nl_', str(unicodedata.normalize('NFKC', text.lower()))))))
    p = [word2idx[c] for c in p.replace('_nl_', '\n') if c in word2idx]
    v = sess.run(model.rnn_outputs, feed_dict={model.inputs: np.expand_dims(np.array(p), -1), model.reset_state: True, model.seq_lens: [len(p)]})
    v = np.squeeze(v)
    me = np.mean(v, axis=0, keepdims=False)
    l = v[-1, :]
    ma = np.max(v, axis=0, keepdims=False)
    return clf.predict(np.expand_dims(np.concatenate((l,me,ma),axis=-1),0))

In [110]:
predict_class('laptop')

array([-1])

In [111]:
for m, p in zip(df2['message'],clf.predict(np.stack(df2['feat']))):
    if p > 0:
        continue
    print(m)
    print(p)
    print('-' * 89)

hoành_tráng_chẳng_khác_gì_những_chiếc_xe_hầm_hố_trên_phim_:N_nl_#genk
-1
-----------------------------------------------------------------------------------------
phần_mềm_test_số_có_bằng_N_không..._why...
-1
-----------------------------------------------------------------------------------------
hôm_nay_trên_#appstore_miễn_phí_khá_nhiều_app_ngon_nhé_các_bác,_có_cả_#remote_mouse_mac,_#adblock_browser_widget,_#media_cast,_#stopad_và_#gif_maker_studio
-1
-----------------------------------------------------------------------------------------
kết_quả_minigame_ngày_N/N/N
-1
-----------------------------------------------------------------------------------------
apple_vs_chinabuye
-1
-----------------------------------------------------------------------------------------
tiền_và_bóng_đá_đúng_là_khó_mà_có_thể_tách_rời_nhau_:(_nl_#genk
-1
-----------------------------------------------------------------------------------------
nhìn_ngứa_mắt_quá_thì_xoá_đi_thôi_:d
-1
----------------------

-1
-----------------------------------------------------------------------------------------
mỗi_ngày_lại_leak_N_tính_năng_sao?_😍_nl_#genk
-1
-----------------------------------------------------------------------------------------
nissan_navara__nl_nhanh_tay_để_là_chủ_sở_hữu_những_chiếc_navara_đầu_tiên_N_tại_việt_nam_nl_lh:N
-1
-----------------------------------------------------------------------------------------
định_luật_moore_có_thể_sắp_chết,_nhưng_sự_sáng_tạo_thì_không!_nl_#genk
-1
-----------------------------------------------------------------------------------------
kết_mỗi_cái_màu_tim_tím
-1
-----------------------------------------------------------------------------------------
đây_cũng_là_minh_chứng_rất_rõ_cho_việc_đừng_nên_quá_chú_trọng_vào_tiền_bạc_mà_mua_N_chiếc_xe_cũ,_nhất_là_cho_con_cái..._nl_#genk
-1
-----------------------------------------------------------------------------------------
hai_hãng_game_nổi_tiếng_sắp_đối_đầu_nhau.
-1
-------------------------------

-----------------------------------------------------------------------------------------
N.N_mp_và_N_tuổi_đời_-_leica_d-lux_classic_là_đời_lai_của_leica_và_panasonic_của_nhựt_bổn_đây_anh_em.
-1
-----------------------------------------------------------------------------------------
sức_mạnh_của_geek_😂_nl_#genk
-1
-----------------------------------------------------------------------------------------
[help]_anh_em_cho_hỏi_học_lập_trình_thì_học_ở_đâu_hà_nội_tốt_với_ạ._cám_ơn_anh_em_nhiều._:)
-1
-----------------------------------------------------------------------------------------
theo_các_bác_galaxy_jN_sẽ_có_giá_bao_nhiêu?
-1
-----------------------------------------------------------------------------------------
lễ_rung_chuông_sẽ_không_còn_diễn_ra..._nl_#genk
-1
-----------------------------------------------------------------------------------------
vai_phản_diện_trong_bộ_phim_được_nói_rằng_hay_hơn_cả_avengers:_infinity_war_sẽ_như_thế_nào?_:N_nl_#genk
-1
-----------------------

-----------------------------------------------------------------------------------------
một_vị_vua_vào_mùa_khô,_nhưng_sẽ_trở_thành...kẻ_vô_gia_cư_vào_mùa_mưa_bão_:v_nl_#genk
-1
-----------------------------------------------------------------------------------------
các_fan_sony_có_thấy_"tăng_nhiệt"_không_:N
-1
-----------------------------------------------------------------------------------------
nhân_họa_đắc_phúc,_hay_vì_tài_năng_biện_luận_của_mark_đã_quá_thần_thánh?_:v_nl_#genk
-1
-----------------------------------------------------------------------------------------
N_xoá_phông
-1
-----------------------------------------------------------------------------------------
theo_bạn_thì_quảng_cáo_này_có_đáng_bị_gỡ_không?
-1
-----------------------------------------------------------------------------------------
sạc_dự_phòng_thì_quá_tiện_rồi,_nhưng_kèm_theo_đó_là_nguy_cơ_cháy_nổ_nếu_sử_dụng_"ẩu".
-1
----------------------------------------------------------------------------------

đã_chính_thức_ra_mắt_và_sắp_về_việt_nam_:d
-1
-----------------------------------------------------------------------------------------
grab_cơm_sườn._buổi_sáng_cần_thơ!
-1
-----------------------------------------------------------------------------------------
nghe_thì_oách_đấy_nhưng_so_với_sN_thì..._😂_nl_#genk
-1
-----------------------------------------------------------------------------------------
cập_nhật_những_tin_tức_công_nghệ_-_lập_trình_hot_nhất_tại:_techtalk.vn
-1
-----------------------------------------------------------------------------------------
nhìn_chung_là_ổn_lắm,_nhưng_không_nghe_nhạc_khi_lái_xe_nha_anh_em,_người_sau_thì_được
-1
-----------------------------------------------------------------------------------------
nói_chung_là_dùng_thư_nhỏ_nhẹ_như_casl_sẽ_giúp_ta_tiết_kiệm_nhiều_thời_gian_để_làm_sản_phẩm,_code_dễ_đọc_hơn
-1
-----------------------------------------------------------------------------------------
máy_tính_giờ_mắc_một_cách_khủng_khiếp,_anh_em_c

thiên_lý_nhãn_:'(_nl_#genk
-1
-----------------------------------------------------------------------------------------
chia_sẻ_về_asus_zenfone_max_plus_mN,_mời_anh_em_tham_gia_cuộc_thi:_https://tinhte.vn/threads/cuoc-thi-review-zenfone-trung-zenfone-max-plus-mN.N/
-1
-----------------------------------------------------------------------------------------
biết_đâu_bất_ngờ_porsche,_land_rover_hay_rolls-royce_làm_xe_máy_thì_trông_nó_sẽ_như_nào?
-1
-----------------------------------------------------------------------------------------
cảnh_báo:_có_spoil_:v_nl_#genk
-1
-----------------------------------------------------------------------------------------
vui_thôi_đừng_vui_quá..._nl_#genk
-1
-----------------------------------------------------------------------------------------
nỗi_khổ_riêng_chỉ_lập_trình_viên_mới_hiểu
-1
-----------------------------------------------------------------------------------------
vui_thôi_đừng_vui_quá_:v_nl_#genk
-1
------------------------------------

In [ ]:
df['class_pred'] = clf.predict(np.stack(df['feat']))

In [ ]:
prob = clf.predict_proba(np.stack(df['feat']))

In [ ]:
df.loc[y_test.index][df['message'].map(lambda x: 'anh_em' in x)]['class'].value_counts()

In [ ]:
i = 0
for (_, row), p in zip(df.iloc[y_test.index].iterrows(), prob[y_test.index]):
    if row['class'] != row['class_pred'] and (row['class'] not in ['fashion', 'lifestyle'] or row['class_pred'] not in ['fashion','lifestyle']):
        i += 1
        print(i)
        print(row['message'])
        print(row['class'], row['class_pred'], p, np.sum(p))
        print('-' * 80)